## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-21-44-35 +0000,nyt,How El Salvador Is Reaping Rewards From Trump’...,https://www.nytimes.com/2025/07/10/world/ameri...
1,2025-07-11-21-41-30 +0000,bbc,Everything you need to know about Wafcon 2024 ...,https://www.bbc.com/sport/football/articles/cq...
2,2025-07-11-21-38-56 +0000,bbc,South Africa rescue point against Tanzania at ...,https://www.bbc.com/sport/football/articles/cr...
3,2025-07-11-21-35-14 +0000,nypost,Skydance reportedly in talks to buy Bari Weiss...,https://nypost.com/2025/07/11/media/skydance-r...
4,2025-07-11-21-33-21 +0000,cbc,Ethics commissioner publishes list of PM Carne...,https://www.cbc.ca/news/politics/mark-carney-f...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2334/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,13
90,canada,6
91,tariff,5
59,rate,4
89,latest,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
35,2025-07-11-13-41-59 +0000,cbc,CUSMA-compliant goods exempt from Trump's late...,https://www.cbc.ca/news/business/trump-tariff-...,36
10,2025-07-11-19-40-14 +0000,nyt,Trump’s Latest Canada Tariff Threats Come Desp...,https://www.nytimes.com/2025/07/11/world/canad...,35
29,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,33
34,2025-07-11-13-46-34 +0000,nypost,"Dow tumbles, S&P 500 slips from record high af...",https://nypost.com/2025/07/11/business/sampp-5...,33
38,2025-07-11-13-13-32 +0000,cbc,RECAP | Trump's latest tariff threat sends tra...,https://www.cbc.ca/news/politics/livestory/rec...,29


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
35,36,2025-07-11-13-41-59 +0000,cbc,CUSMA-compliant goods exempt from Trump's late...,https://www.cbc.ca/news/business/trump-tariff-...
6,20,2025-07-11-20-46-00 +0000,wsj,A dispute over the Federal Reserve’s renovatio...,https://www.wsj.com/economy/central-banking/je...
29,20,2025-07-11-14-53-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
24,18,2025-07-11-16-28-54 +0000,nypost,NBC’s ‘Nightly News’ with Tom Llamas beats ABC...,https://nypost.com/2025/07/11/media/nbcs-night...
19,15,2025-07-11-17-37-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
50,15,2025-07-10-23-27-50 +0000,nypost,Jamie Dimon lashes out at ‘Marxist’ Zohran Mam...,https://nypost.com/2025/07/10/business/jamie-d...
20,14,2025-07-11-17-14-01 +0000,nypost,Unilever appoints new Ben & Jerry’s CEO as bat...,https://nypost.com/2025/07/11/business/unileve...
54,14,2025-07-10-22-07-39 +0000,nypost,"Tennis legends Andy Roddick, Martina Navratilo...",https://nypost.com/2025/07/10/business/tennis-...
3,13,2025-07-11-21-35-14 +0000,nypost,Skydance reportedly in talks to buy Bari Weiss...,https://nypost.com/2025/07/11/media/skydance-r...
11,13,2025-07-11-19-33-00 +0000,nypost,Reddit flooded with antisemitic conspiracy the...,https://nypost.com/2025/07/11/business/reddit-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
